In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV

from sklearn.feature_selection import mutual_info_regression

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

from sklearn.metrics import mean_squared_error, r2_score, make_scorer

import klib

from lightgbm import LGBMRegressor

from joblib import dump, load

In [2]:
df = pd.read_csv(r"C:\Users\Saidabrorkhon\ML_Lectures\ML 3rd month\song_data.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18835 entries, 0 to 18834
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_name         18835 non-null  object 
 1   song_popularity   18835 non-null  int64  
 2   song_duration_ms  18835 non-null  int64  
 3   acousticness      18835 non-null  float64
 4   danceability      18835 non-null  float64
 5   energy            18835 non-null  float64
 6   instrumentalness  18835 non-null  float64
 7   key               18835 non-null  int64  
 8   liveness          18835 non-null  float64
 9   loudness          18835 non-null  float64
 10  audio_mode        18835 non-null  int64  
 11  speechiness       18835 non-null  float64
 12  tempo             18835 non-null  float64
 13  time_signature    18835 non-null  int64  
 14  audio_valence     18835 non-null  float64
dtypes: float64(9), int64(5), object(1)
memory usage: 2.2+ MB


In [4]:
df.head(3)

,song_name,song_popularity,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
0,Boulevard of Broken Dreams,73,262333,0.00552,0.496,0.682,0.000029,8,0.0589,-4.095,1,0.0294,167.060,4,0.474
1,In The End,66,216933,0.01030,0.542,0.853,0.000000,3,0.1080,-6.407,0,0.0498,105.256,4,0.370
2,Seven Nation Army,76,231733,0.00817,0.737,0.463,0.447000,0,0.2550,-7.828,1,0.0792,123.881,4,0.324


In [5]:
df = klib.data_cleaning(df)

Shape of cleaned data: (14926, 15) - Remaining NAs: 0


Dropped rows: 3909
     of which 3909 duplicates. (Rows (first 150 shown): [173, 179, 212, 414, 421, 423, 424, 463, 471, 477, 606, 610, 655, 662, 663, 664, 667, 672, 676, 677, 680, 681, 682, 683, 685, 686, 687, 688, 689, 690, 698, 700, 704, 719, 721, 723, 732, 734, 736, 737, 743, 763, 764, 767, 772, 788, 798, 804, 808, 817, 820, 827, 828, 829, 830, 832, 841, 950, 958, 970, 990, 994, 1022, 1025, 1030, 1057, 1061, 1064, 1066, 1077, 1080, 1081, 1090, 1100, 1104, 1105, 1108, 1114, 1116, 1118, 1120, 1122, 1126, 1128, 1131, 1132, 1139, 1164, 1181, 1241, 1271, 1277, 1307, 1315, 1317, 1322, 1454, 1462, 1557, 1559, 1563, 1566, 1569, 1571, 1573, 1579, 1581, 1596, 1705, 1708, 1744, 1749, 1814, 1824, 1825, 1826, 1832, 1843, 1848, 1850, 1868, 1869, 1905, 1907, 1917, 1918, 1920, 1923, 1925, 1935, 1939, 1945, 1949, 1956, 2039, 2042, 2043, 2060, 2076, 2110, 2114, 2115, 2119, 2123, 2126, 2127, 2128, 2129, 2130, 2140])

Dropped columns: 0
     of w

In [16]:
num_col = x.select_dtypes(include='number').columns
cat_col = x.select_dtypes(exclude='number').columns

In [17]:
x = df.drop(columns=['song_popularity'])
y = df['song_popularity']

In [18]:
mi = mutual_info_regression(df[num_col],y)
mdf = pd.DataFrame(mi, index=df[num_col].columns).sort_values(by=[0],ascending=False)

In [9]:
mdf

,0
song_popularity,4.334922
instrumentalness,0.033554
loudness,0.025515
acousticness,0.014757
audio_mode,0.010007
audio_valence,0.007852
time_signature,0.007825
speechiness,0.007596
energy,0.005853
song_duration_ms,0.005713


In [10]:
df.drop(columns=['key', 'audio_mode','time_signature'], inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14926 entries, 0 to 14925
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_name         14926 non-null  string 
 1   song_popularity   14926 non-null  int8   
 2   song_duration_ms  14926 non-null  int32  
 3   acousticness      14926 non-null  float32
 4   danceability      14926 non-null  float32
 5   energy            14926 non-null  float32
 6   instrumentalness  14926 non-null  float32
 7   liveness          14926 non-null  float32
 8   loudness          14926 non-null  float32
 9   speechiness       14926 non-null  float32
 10  tempo             14926 non-null  float32
 11  audio_valence     14926 non-null  float32
dtypes: float32(9), int32(1), int8(1), string(1)
memory usage: 714.4 KB


In [12]:
# cat_col = df.select_dtypes(exclude='number').columns
# le = LabelEncoder()
# for col in cat_col:
#   df[col] = le.fit_transform(df[col])

# num_col = df.select_dtypes(include='number').columns

# scaler = StandardScaler()
# df[num_col] = scaler.fit_transform(df[num_col])
# df.head(2)

In [19]:
num_transformer = Pipeline(steps=[
  ('imputer', SimpleImputer(strategy='mean')),
  ('scaler', StandardScaler())
])

cat_transformer = Pipeline(steps=[
  ('imputer', SimpleImputer(strategy='most_frequent')),
  ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
  ('num', num_transformer, num_col),
  ('cat', cat_transformer, cat_col)
])

In [20]:
x = df.drop(columns=['song_popularity'])
y = df['song_popularity']

x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.2, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

In [27]:
model = LGBMRegressor(random_state=42)

pipeline = Pipeline(steps=[
  ('preprocessor', preprocessor),
  ('model', LGBMRegressor(n_estimators=100, random_state=42))
])

In [28]:
pipeline.fit(x_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001549 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 11940, number of used features: 10
[LightGBM] [Info] Start training from score 48.792379


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['song_duration_ms', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo',
       'audio_valence'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['song_name'], dtype='object'))])),
                ('model', LGBMRegressor(random_state=42))])

In [29]:
y_pred = pipeline.predict(x_val)

c:\Users\Saidabrorkhon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [30]:
mse = mean_squared_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)
print(mse)
print(r2)

395.20682157396044
0.033178839097712465


In [31]:
model = pipeline
dump(model, 'song_popularity.joblib')

['song_popularity.joblib']

In [32]:
model = load('song_popularity.joblib')

In [33]:
x_test.to_csv('song_x_test.csv', index=False)
y_test.to_csv('song_y_test.csv', index=False)